In [0]:
# all imports in here
import torch
import numpy as np
import random
from scipy.special import softmax

In [0]:
def gen_attack(N, x_orig, t, delta_max, alpha,rho, tau, G):
    # Algo 1 from : https://arxiv.org/pdf/1805.11090.pdf
    """
        N : size of population
        x_orig: original example
        t: target label
        delta_max: maxmimum distance 
        alpha: mutation range 
        rho: mutation probability (~ 1% - 3%)
        tau: sampling temperature
        G: # of generations
    """
    # initialize population
    population = []
    for i in population:
        new_random_member = random_matrix(x_orig.shape())
        population.append(new_random_member + x_orig)

    for g in range(G):
        for i in population:
            # compute fitness
    pass    



In [0]:
def mutation_op(cur_pop, idx, step_noise=0.01, p=0.005):
    """
        cur_pop: the current population
        idx    : an index. useful for debugging
        step_noise: the bound on the noise to construct
        p: proportion of elements in the current population to perturb
    """
    perturb_noise = torch.FloatTensor(cur_pop.shape).uniform_(-step_noise, step_noise)
    mask = (torch.FloatTensor(cur_pop.shape) < p).double()
    mutated_pop = perturb_noise * mask + cur_pop
    return mutated_pop

x = torch.FloatTensor(2, 2)
print(x)
print(mutation_op(x, 1, step_noise=0))

tensor([[1.6351e-35, 0.0000e+00],
        [4.4842e-44, 0.0000e+00]])
tensor([[1.6351e-35, 0.0000e+00],
        [4.4842e-44, 0.0000e+00]], dtype=torch.float64)


In [0]:
y = torch.FloatTensor(10,1).uniform_(0,1)
y /= torch.sum(y)
print(y)
print(y[1])
print(y[-1])

tensor([[0.0489],
        [0.1979],
        [0.0197],
        [0.1468],
        [0.0495],
        [0.0838],
        [0.1296],
        [0.1822],
        [0.0039],
        [0.1378]])
tensor([0.1979])
tensor([0.1378])


In [0]:
def fitness(model, x, target_class):
    """
        model: a pytorch neural network model (takes x to a probability vector)
        x:     a single image, or adversarial example
        target_class: the class label of x (as an integer)
        This source code is inspired by:
        https://github.com/nesl/adversarial_genattack/blob/2304cdc2a49d2c16b9f43821ad8a29d664f334d1/genattack_tf2.py#L39
    """
    # TODO: make sure this is compatible with the NN,
    #       i.e., is probs indexable? or is it just a class
    with torch.no_grad():
        probs = model(x)
        log_probs = torch.log(probs)
        f = 2 * log_probs[target_class] - torch.sum(log_probs)
        return f

    

In [0]:
def selection(fitness_fun, population, model, data, target, idx, num_elite = 2, pop_size = 10):
  """
  Input
  fitness_fun: the objective function by which we're using to calculate the
               the fitness - in this case it's the loss function
  population: the population of individuals of which to select the number for
              the next generation
  model: the PyTorch NN model 
  data: the input value to find a perturbation for
  target: the actual class of the input value
  idx: the generation number we're on - for debugging purposes
  num_elite: the number of elites to carry on from the previous generations
  pop_size: the size of each population
  Output
  new_pop: Returns the new population of individuals
  """
  #Find fitness for every individual in the population
  fitness = []
  for ind in population:
    #Take original data and add pertubation from GA individual
    x = data + ind
    data_fit = fitness_fun(model, x, target)
    #Append the perturbed data and the fitness value to the list
    fitness.append(x, data_fit)

  #List of individual and their proportional fitness
  tot_fit = sum(x[1] for x in fitness)
  prob =  [x[1]/tot_fit for x in fitness]
  data_ind = [x[0] for x in fitness]
  fit = list(zip(data_ind, prob))
  #Sort the list by decreasing order - largest fitness first
  fit.sort(key=lambda x: x[1], reverse=True)

  #Initialize new population by adding the elites
  new_pop = fit[:num_elite]
  #Use this line if we only want to add the data, no fitness
  new_pop = [x[0] for x in new_pop] 

  #Create roulette wheel to get parents
  a = min(fit)[1] #min bound 
  b = max(fit)[1] #max bound
  i = 0
  parents = []

  #Need 2*(pop_size - num_elite) parents
  #Already n elites, the rest are children and need two parents for one child
  while i < 2*(pop_size-num_elite): #Fill parents until it's the desired size
    t = (b-a)*np.random.rand(1) + a  #The value to see if the current parent is chosen
    j = random.choice(fit) #Picks a random tuple from the list

    if j[1] > t: #If fitness is greater than t add to parents - This is what assures that less fit solutions are less likely to be picked
      parents.append(j[0]) #Pick parent - only append data (can change if we do want fitness)
      i += 1

    #Will have duplicates, but an individual can mate more than once
  k = 0
  child_pop = []
  while k < pop_size:
    #Get two parents next to each other and get their child
    p1 = parents[k]
    p2 = parents[k+1]
    child_pop.append(crossover(p1, p2))
    k += 2
    
  #Take the new children and mutate them - add mutated children to new_pop
  new_pop.extend(mutation_op(child_pop, idx))
  
  return new_pop


In [3]:
ll = []
for i in range(10):
  ll.append((i, i*2))
print(random.choice(ll))

(6, 12)


In [0]:
fitness = []
for i in range(10):
  fitness.append((i, i+2))

totfit = sum(x[1] for x in fitness)
prob =  [x[1]/totfit for x in fitness]
data = [x[0] for x in fitness]
fit = list(zip(data, prob))
fit.sort(key=lambda x: x[1], reverse=True)


In [49]:
j = random.choice(fit)
print(j)

(0, 0.03076923076923077)


In [0]:
def crossover(parent1, parent2):
  """
  Point selection cross-over
  From 0-num is parent 1, from num-end is parent 2
  Implemented so num can never be 0 or end
  input
  parent1: individual in old population
  parent2: individual in old population
  output
  child: new individual from mating of parents
  """
  n = len(parent1) - 1
  cx = np.random.randint(1, n-1) #choose a random crossover point
  child = parent1[:cx]+parent2[cx:]
  return child

In [0]:
#def crossover(parent1, parent2):
  #p = Fit(p1)/Fit(p1)+Fit(p2)

  # For each element
  # randnum = Uniform(0, 1)
  # choose element from p1 is randnum >=p
  # choose element form p2 otherwise
  #return child